In [ ]:
import numpy as np
import struct 
import os
import plotly.graph_objects as go

import lgchimera.general as general
from lgchimera.io import read_lidar_bin, read_gt

%load_ext autoreload
%autoreload 2

Read lidar points

In [ ]:
binpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', '2011_09_29_drive_0071_sync', 'velodyne_points', 'data')
PC_data = read_lidar_bin(binpath)

In [ ]:
N = len(PC_data)

In [ ]:
# fig = go.Figure(data=general.pc_plot_trace(PC_data[0], size=2))
# fig.update_layout(width=1600, height=900, 
#     scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
# fig.show()

Ground truth trajectory

In [ ]:
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', '2011_09_29_drive_0071_sync', 'oxts', 'data')
gt_data = read_gt(gtpath)
lla = gt_data[:,:3]

In [ ]:
# fig = go.Figure(data=go.Scatter(x=lla[:,0], y=lla[:,1]))
# fig.update_layout(width=900, height=700, 
#     scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
# fig.show()

LiDAR odometry with ICP

In [ ]:
import open3d as o3d
from lgchimera.registration import initialize_source_and_target, p2p_ICP, p2pl_ICP

In [ ]:
from scipy.spatial.transform import Rotation as R

heading = gt_data[0][5] # heading angle
r = R.from_euler('XYZ', [0, 0, heading])
R_heading = r.as_matrix()

In [ ]:
N = len(PC_data)
R_abs = R_heading
t_abs = np.zeros(3)
poses = N * [None]
poses[0] = (R_abs.copy(), t_abs.copy())

lidar_Rs = []
lidar_ts = []

for i in range(1,N):
    print(i)
    trans_init = np.eye(4)
    threshold = 1
    source, target = initialize_source_and_target(PC_data[i], PC_data[i-1])
    reg_p2p, eval_time = p2pl_ICP(source, target, threshold, trans_init)
    R_hat = reg_p2p.transformation[:3,:3]
    t_hat = reg_p2p.transformation[:3,3]

    lidar_Rs.append(R_hat)
    lidar_ts.append(t_hat)

    t_abs += (R_abs @ t_hat).flatten()
    R_abs = R_hat @ R_abs
    poses[i] = (R_abs.copy(), t_abs.copy())

In [ ]:
# TODO: Save registration results to file

In [ ]:
positions = np.zeros((N,3))
for i in range(N):
    positions[i] = poses[i][1]

In [ ]:
lidar_traj = go.Scatter(x=positions[:,0], y=positions[:,1], hovertext=np.arange(N), name='Lidar odometry')
# fig = go.Figure(data=lidar_traj)
# fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
# fig.show()

In [ ]:
from lgchimera.general import lla_to_ecef, ecef2enu

ref_lla = lla[0]

ecef = lla_to_ecef(*lla[0])

gt_ecef = np.zeros((N,3))

for i in range(N):
    ecef = lla_to_ecef(*lla[i])
    gt_ecef[i] = ecef2enu(ecef[0], ecef[1], ecef[2], ref_lla[0], ref_lla[1], ref_lla[2])

gt_ecef = gt_ecef[:,[1,0,2]]

gt_traj = go.Scatter(x=gt_ecef[:,0], y=gt_ecef[:,1], hovertext=np.arange(N), name='Ground-truth')
# fig = go.Figure(data=gt_traj)
# fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
# fig.show()

In [ ]:
fig = go.Figure(data=[gt_traj, lidar_traj])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
gt_traj3d = go.Scatter3d(x=gt_ecef[:,0], y=gt_ecef[:,1], z=gt_ecef[:,2], marker=dict(size=2), hovertext=np.arange(N), name='Ground-truth')
lidar_traj3d = go.Scatter3d(x=positions[:,0], y=positions[:,1], z=positions[:,2], marker=dict(size=2), hovertext=np.arange(N), name='Lidar odometry')
fig = go.Figure(data=[gt_traj3d, lidar_traj3d])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

## Factor Graph test

Lidar odometry and simulated GPS with noisy ground-truth

In [ ]:
from lgchimera.pose_graph import PoseGraph

g = PoseGraph()

# Add initial node
R_abs = R_heading.copy()
t_abs = gt_ecef[0].copy()
g.add_node(1, (R_abs, t_abs))

# GPS noise parameters
gps_sigma = 0.2  # [m]

# ICP parameters
trans_init = np.eye(4)
threshold = 1

# For each new frame
for i in range(1,50):
    print(i)

    # Perform LiDAR odometry
    source, target = initialize_source_and_target(PC_data[i], PC_data[i-1])
    reg_p2p, eval_time = p2pl_ICP(source, target, threshold, trans_init)
    R_hat = np.array(reg_p2p.transformation[:3,:3])
    t_hat = np.array(reg_p2p.transformation[:3,3])

    # Initialize new node with LiDAR odometry estimate
    R_abs = R_hat @ R_abs
    t_abs += R_abs @ t_hat
    g.add_node(i+1, (R_abs, t_abs))

    # Add LiDAR odometry edge
    g.add_edge([i, i+1], (R_hat, t_hat))

    # Simulate GPS measurement
    gps_pos = gt_ecef[i] + np.random.normal(0, gps_sigma, 3)

    # Add GPS factor and edge
    g.add_factor(i+1, (np.eye(3), gps_pos))

In [ ]:
fig = go.Figure(data=g.plot_trace())
fig.update_layout(width=1600, height=900, scene=dict(aspectmode='data'))
fig.show()

## Add Authentication